## Finger Detection and Counting

In [1]:
#Steps
#Grab ROI
#Calculate a running average background value for 60 frames of video
#Img avg value is found, then the hand can enter the ROI
#Once hand enter ROI, we can detect change and apply thresholding
#In the ROI, use convex hull to draw polygon around the hand
#Calculate center of the hand against the angle of outer points to infer finger count

In [2]:
import cv2
import numpy as np

# Used for distance calculation
from sklearn.metrics import pairwise

In [3]:
#Global variables
# Background is a global variable that will update through a few functions
background = None

# Halfway point between 0 and 1 of accumulated weight
accumulated_weight = 0.5

# Set up the ROI for grabbing the hand.
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [4]:
#Finding Average Background Value
#The function calculates the weighted sum of the input image src and
#the accumulator dst so that dst becomes a running average of a frame sequence:

def calc_accum_avg(frame, accumulated_weight):
    
    global background
    
    # create the background from a copy of the frame.
    if background is None:
        background = frame.copy().astype("float")
        return None

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(frame, background, accumulated_weight)

## Segment the Hand Region in Frame

In [5]:
def segment(frame, threshold=25):
    global background
    
    # Calculates the Absolute Differentce between the backgroud and the passed in frame
    diff = cv2.absdiff(background.astype("uint8"), frame)

    # Applying threshold to the image
    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # Grabbing external contours form the image
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If length of contours list is 0, then no contours were grabbed
    if len(contours) == 0:
        return None
    else:
        # Assuming the largest external contour in ROI is the hand
        hand_segment = max(contours, key=cv2.contourArea)
        
        # Return hand segment and thresholded hand image
        return (thresholded, hand_segment)

In [6]:
def count_fingers(thresholded, hand_segment):
    
    # Calculated the convex hull of the hand segment
    conv_hull = cv2.convexHull(hand_segment)
    
    #getting convex hull 4 most outward points, on the top, bottom, left, and right by using argmin and argmax.
   
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])

    # Assuming the center of the hand is half way between the top and bottom and halfway between left and right
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2

    # maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull
    # Euclidean Distance between the center of the hand and the left, right, top, and bottom.
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    
    #largest distance
    max_distance = distance.max()
    
    #circle with 90% radius of the max euclidean distance
    radius = int(0.8 * max_distance)
    circumference = (2 * np.pi * radius)

    #ROI of only that circle
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    
    # drawing circular ROI
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    # Using bit-wise AND with the cirle ROI as a mask.
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  
    count = 0

    for cnt in contours:
        
        # Bounding box of countour
        (x, y, w, h) = cv2.boundingRect(cnt)

        # Increment count of fingers based on two conditions:
        # 1. Contour region is not the very bottom of hand area (the wrist)
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        
        # 2. Number of points along the contour does not exceed 25% of the circumference of the circular ROI (otherwise we're counting points off the hand)
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        if  out_of_wrist and limit_points:
            count += 1

    return count

## Run Program

In [8]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
   
    ret, frame = cam.read()

    #flipping the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # For the first 30 frames, calculate the average of the background.
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "bg avg", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
            
    else:
      
        # segmenting the hand region
        hand = segment(gray)

        if hand is not None:
        
            thresholded, hand_segment = hand

            # Drawing contours around hand segment, counting and displaying images
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)
            fingers = count_fingers(thresholded, hand_segment)
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Thesholded", thresholded)

    # Draw ROI Rectangle on frame copy
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)
    num_frames += 1

    # Display the frame with segmented hand
    cv2.imshow("Finger Count", frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()